In [60]:
import plotly as plt
import plotly.graph_objs as go
import numpy as np
import pandas as pd

In [61]:
usda = pd.read_csv('All Dealers.csv')
benji = pd.read_csv("Missouri.csv")
usDF = pd.DataFrame(usda)
benjiDF = pd.DataFrame(benji)
print("Head of usda all Data")
print(usDF.head())
print("----------------------\n Head of benji Data")
print(benjiDF.head())

Head of usda all Data
  First Name    Last Name                  Kennel Name        City     State  \
0      Elisa     Brandvik                          NaN       Ozark  Arkansas   
1      Frank       Foster                          NaN  Hindsville  Arkansas   
2    Shaqwan       Hagans      Good Life Luxury Kennel   Covington   Georgia   
3    Stephen     Thurmond        Little Heart’s Kennel       Comer   Georgia   
4      Donna  Vanderzwaag  Byron Christian Academy LLC       Byron   Georgia   

       USDA License #    State License #  \
0           71-A-1423                NaN   
1  71-A-1408 (Former)                NaN   
2                 NaN      36118802 (ID)   
3                 NaN            3700107   
4                 NaN  36104424 (Former)   

                              All Inspection Reports  \
0  PST_Inspection_Report_Elisa Brandvik (10).pdf ...   
1  PST_Teachable_Moments_FRANK FOSTER_06242022 (1...   
2  GoodLifeLuxuryKennel7.27.2022.pdf (https://v5....   
3  Littl

In [62]:
#Filter for only missouri, some data cleaning
missouriBenjiData = benjiDF[benjiDF["State"] == "Missouri"]
print("List of counties in Benji:\n", missouriBenjiData["County"])
print(missouriBenjiData["State"])
print("Before NA transform")
print(missouriBenjiData.head())
missouriBenjiData = benjiDF.fillna(0)
missouriBenjiData.head()


List of counties in Benji:
 0            Clark
1       St Charles
2            Barry
3              NaN
4             Knox
           ...    
1475      Lawrence
1476         Barry
1477        Wright
1478        Newton
1479        Jasper
Name: County, Length: 1480, dtype: object
0       Missouri
1       Missouri
2       Missouri
3       Missouri
4       Missouri
          ...   
1475    Missouri
1476    Missouri
1477    Missouri
1478    Missouri
1479    Missouri
Name: State, Length: 1480, dtype: object
Before NA transform
                                  Name                    Kennel Name  \
0  Nathan Schreiber\nJuanita Schreiber  Circle S, Puppies and Paints,   
1                         Campbell Bee                            NaN   
2                           Bob Hughes      Southwest Auction Service   
3                Henry A. Grosenbacher                            NaN   
4                       Michael Franke             North River Kennel   

            Address        City   

,Name,Kennel Name,Address,City,County,State,Adult Count,Offspring Count,License Type,Current USDA,...,Pet Store (from Field 23),State (from Field 23),Status (from Field 23),Pet Store Connections 2023,Pet Store (from Pet Store Connections 2023),State (from Pet Store Connections 2023),Online Brokers,Status (from Pet Store Connections 2023),Field 43,Additional Information
0,Nathan Schreiber\nJuanita Schreiber,"Circle S, Puppies and Paints,",0,Revere,Clark,Missouri,0,0,?,0,...,0,0,0,0,0,0,0,0,0.0,0
1,Campbell Bee,0,3262 Meeker St,St Charles,St Charles,Missouri,0,0,animals confiscated,0,...,0,0,0,0,0,0,0,0,0.0,0
2,Bob Hughes,Southwest Auction Service,"Po Box 534,",Wheaton,Barry,Missouri,219,142,Auction House,43-B-0386,...,0,0,0,0,0,0,0,0,0.0,0
3,Henry A. Grosenbacher,0,PO Box 616,Cabool,0,Missouri,0,0,Auction House,43-B-3473,...,0,0,0,0,0,0,0,0,0.0,0
4,Michael Franke,North River Kennel,65723 St Hwy 151,Novelty,Knox,Missouri,105,90,Breeder,0,...,0,0,0,0,0,0,0,0,0.0,0


In [63]:
#geocode and get coordinates for county and state for the purpose of visualization
#!pip install geopy
from geopy.geocoders import Nominatim
from time import sleep
geolocator = Nominatim(user_agent="animalAidAnalysis")
#test the geolocator with the second row (row 1)
testGeocode = missouriBenjiData.iloc[1]
testGeoAddress = testGeocode["Address"]
testGeoCity = testGeocode["City"]
testGeoCounty = testGeocode["County"]
testGeoState = testGeocode["State"]

#sleeo is necessary due to how the api works
geoCordinatesTest = geolocator.geocode(testGeoAddress + ", " + testGeoCity + ", " + testGeoCounty + ", " + testGeoState)
print(f"Test geocoding for : {testGeoAddress}, {testGeoCity}, {testGeoCounty} {testGeoState}")
print(f"Coordinates: {geoCordinatesTest.latitude}, {geoCordinatesTest.longitude}")

location = f'{geoCordinatesTest.latitude},{geoCordinatesTest.longitude}'
reversedTest = geolocator.reverse(location)
address = reversedTest.raw["address"]
city = address["city"]
county = address["county"]
state = address["state"]

print(f"City: {city}")
print(f"County: {county}")
print(f"State: {state}")


Test geocoding for : 3262 Meeker St, St Charles, St Charles Missouri
Coordinates: 38.83737075887174, -90.5051742969412
City: Saint Charles
County: Saint Charles County
State: Missouri


In [77]:
import requests
from time import sleep
def getCoords(row, api_key):
    rowAddress = str(row["Address"])
    rowCity = str(row["City"])
    rowCounty = str(row["County"])
    rowState = str(row["State"])
    if address != 0:
      url = f'https://us1.locationiq.com/v1/search.php?key={api_key}&q={rowAddress}, {rowCity}, {rowCounty}, {rowState}&format=json'
    else:
      url = f'https://us1.locationiq.com/v1/search.php?key={api_key}&q={rowCity}, {rowCounty}, {rowState}&format=json'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data:
                return {
                    'latitude': float(data[0]['lat']),
                    'longitude': float(data[0]['lon'])
                }
            else:
                return None
        else:
            print(f'Error: {response.status_code}')
            return None
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

# Replace 'YOUR_API_KEY' with your actual LocationIQ API key
api_key = 'pk.224e8d22dee0cfe54b9cf3070392d434'

# Initialize Latitude and Longitude columns
missouriBenjiData["Latitude"] = pd.NA
missouriBenjiData["Longitude"] = pd.NA

# Iterate over the DataFrame to get coordinates
for i, row in missouriBenjiData.iterrows():
    coordinates = getCoords(row, api_key)
    if coordinates:
        missouriBenjiData.at[i, "Latitude"] = coordinates['latitude']
        missouriBenjiData.at[i, "Longitude"] = coordinates['longitude']
        print(f'Geocoding successful for row {i}: {row["Address"]}, {row["City"]}, {row["County"]}, {row["State"]}')
    else:
        print(f'Geocoding failed for row {i}: {row["Address"]}, {row["City"]}, {row["County"]}, {row["State"]}')

    sleep(1)
print(missouriBenjiData)

Geocoding successful for row 0: 0, Revere, Clark, Missouri
Geocoding successful for row 1: 3262 Meeker St, St Charles, St Charles, Missouri
Geocoding successful for row 2: Po Box 534, , Wheaton, Barry, Missouri
Geocoding successful for row 3: PO Box 616, Cabool, 0, Missouri
Geocoding successful for row 4: 65723 St Hwy 151, Novelty, Knox, Missouri
Geocoding successful for row 5: RR 1 Box 370, , Ava, Douglas , Missouri
Geocoding successful for row 6: 1662 West State Hwy H, Shell Knob, Stone, Missouri
Geocoding successful for row 7: 378 Leadmine Rd,, Tunas, Dallas , Missouri
Geocoding successful for row 8: 4311 W Hwy 86, Joplin, Jasper, Missouri
Geocoding successful for row 9: 20976 Rye Ln, Richland, Pulaski, Missouri
Geocoding successful for row 10: 100 S. Blackstone, Malta Bend, Saline , Missouri
Geocoding successful for row 11: 9801 East 20th , Mountain Grove, Texas, Missouri
Geocoding successful for row 12: 12325 County Road 6590, West Plains, Howell, Missouri
Geocoding successful for

In [78]:
print(missouriBenjiData)

                                     Name                    Kennel Name  \
0     Nathan Schreiber\nJuanita Schreiber  Circle S, Puppies and Paints,   
1                            Campbell Bee                              0   
2                              Bob Hughes      Southwest Auction Service   
3                   Henry A. Grosenbacher                              0   
4                          Michael Franke             North River Kennel   
...                                   ...                            ...   
1475                          Leon Miller                              0   
1476                       Jera Tilhendor                              0   
1477                    Stephen Stoltzfus                              0   
1478       Chris Bailey \nChristin Bailey          Bailey Transportation   
1479                       Robert Johnson       Johnson's Precious Cargo   

                  Address          City      County     State Adult Count  \
0         

In [80]:
missouriBenjiData.to_csv("geoCodedBenjiData.csv")

In [89]:
import plotly.express as px

fig = px.scatter_mapbox(missouriBenjiData, lat="Latitude", lon="Longitude", hover_name="City", hover_data=["Adult Count","Offspring Count","License Type"],
                        color_discrete_sequence=["fuchsia"], zoom=5, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()